In [ ]:
import os
import sys
from pathlib import Path
import datetime
import rasterio as rio
from rasterio import plot
from rasterio.plot import show
from rasterio.merge import merge
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches

#import shutil
#import tempfile
#import json
#import random
import numpy as np
import pandas as pd
#import geopandas as gpd
#from pyproj import Proj, transform
#from pyproj import CRS
#from shapely.geometry import box
#from shapely.geometry import shape
#from shapely.geometry import MultiPoint
#from shapely.geometry import Point
#from shapely.geometry import Polygon
#import xarray as xr

%matplotlib inline

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n gridCell = {} \n index_dir = {} \n local_dir = {}"
      .format(basic_config['grid_cell'],basic_config['index_dir'],basic_config['local_dir']))

%store -r single_output_params
print("SingleOutputParams: \n ClassPrefix = {} \n ClassFile = {}"
      .format(single_output_params['class_prefix'],single_output_params['class_file']))

## Define classification system

In [ ]:
##TODO: Make dictionaries for each classification option
LC_labels = ["bare/built",
            "water",
            "lowveg",
            "medVeg",
            "plantation",
            "mature forest"]
colors = ['linen',
          'blue',
          'lightgreen',
          'yellowgreen',
          'purple',
          'darkgreen'
         ]
cmap = ListedColormap(colors)

patches = []
for lc in range(len(LC_labels)):
    patch = mpatches.Patch(color=colors[lc], label=LC_labels[lc])
    patches.append(patch)

class_bins = [1, 7, 10, 40, 60, 80,100]
norm = BoundaryNorm(class_bins,
                    len(colors))

## Get band info

In [ ]:
with rio.open(single_output_params['class_file']) as src:
    stack = src.read()
    num_bands = stack.shape[0]
    
##Get band years from input data (assumes that data in selected index directory corresponds to that used in classification)
##TODO: make option of inputing own start year instead.

yr_stack = []
print("Getting year list from: {}".format(basic_config['index_dir']))
for r in os.listdir(basic_config['index_dir']):
    if r.endswith('.tif'):
        in_yr = r[:4]
        if in_yr not in yr_stack:
            yr_stack.append(in_yr)
yr_stack.sort()
##remove the first element in the list (this is the padding and probably not in the final; heck if this is always the case)
print("skipping first year in TS (used as padding)")
yr_stack.pop(0)
stack_data = pd.DataFrame({'MapYear':yr_stack})
stack_data.index.name='Band'
print('Bands in classified output are:')
print(stack_data.T)

## plot classified map at multiple timepoints

In [ ]:
tot_plots = len(single_output_params['map_years'])
cols = 2
rows = tot_plots // cols 
rows += tot_plots % cols
position = range(1,tot_plots + 1)

fig = plt.figure(1, figsize=(20,20))
fig.legend(handles=patches, loc='center')

for k in range(tot_plots):
    ax = fig.add_subplot(rows,cols,position[k])
    with rio.open(single_output_params['class_file']) as src:
        class_yr = src.read(k+1)
    ax.imshow(class_yr, cmap=cmap, norm=norm)
    ax.set_title("Year {}".format(basic_config['yr_range'][k]))
    ax.axis('off')

## view change in plantations over time

## view change in natural forest over time

## mosaic multiple cells

In [ ]:
gridcells = [1157,1158]  #TODO: Make this a parameter
src_to_mosaic = []
for g in gridcells:
    classfile = os.path.join(basic_config['smooth_dir'],'{:06d}'.format(g),'cls',single_output_params['class_prefix']+'{:06d}'.format(g)+'.tif')
    src = rio.open(ClassFile)
    src_to_mosaic.append(src[1])
mosaic, out_trans = merge(src_to_mosaic)
show(mosaic, cmap=cmap)

## view alongside actual image